# Aprendizado de Máquina - Projeto 1
### Guilherme Pereira Corrêa 198397
### Bruno Moreira...

## Parte 1 - Métodos de clusterização

### Coleta e tratamento dos dados

### k-means

### Outro método

## Parte 2 - Redução de dimensionalidade